## measure the trade efficiency of the clusters
after clustering, dataframe stats are delivered on how well a cluster finishes a trade.

In [1]:
import pycaret
from pycaret.clustering import *
import pandas as pd
import datetime

#get most recent clustering element, or specify by name
fn = 'clustered/clustered_kmeans _2023-08-14 09:31:34.csv'
cluster = pd.read_csv(fn)
cluster.head()

,Unnamed: 0,group,time,s_MP,change,type,length,sum_change,s_area,surge_area,...,p_type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,p_length,p_sum_change,p_area,Cluster
0,0,1,1.660222e+12,29.99,0.317588,surge,1,0.317588,0.317588,0.317588,...,precursor,30.00,7349599.5,1.229796e+09,716681.94,314676.40,6.0,-0.004943,-0.029656,Cluster 2
1,1,3,1.660222e+12,29.85,0.000469,surge,1,0.000469,0.000469,0.000469,...,precursor,29.90,7307990.0,1.229792e+09,715352.90,314642.84,1.0,0.000402,0.000402,Cluster 2
2,2,5,1.660222e+12,29.87,0.001105,surge,2,0.001607,0.003214,0.003214,...,precursor,29.92,7285245.0,1.229754e+09,714476.10,313576.00,4.0,-0.002773,-0.011090,Cluster 2
3,3,7,1.660222e+12,29.89,0.001610,surge,1,0.001610,0.001610,0.001610,...,precursor,29.88,7318707.0,1.229744e+09,715730.56,313546.60,2.0,-0.000200,-0.000400,Cluster 2
4,4,9,1.660223e+12,29.90,0.001372,surge,1,0.001372,0.001372,0.001372,...,precursor,29.93,7280582.0,1.229794e+09,714140.10,314889.38,7.0,-0.013670,-0.095690,Cluster 2


,Unnamed: 0,group,time,s_MP,change,type,length,sum_change,s_area,surge_area,...,p_type,p_MP,p_buyCap,p_askCap,p_totalBidVol,p_totalAskVol,p_length,p_sum_change,p_area,Cluster
0,0,1,1.660222e+12,29.99,0.317588,surge,1,0.317588,0.317588,0.317588,...,precursor,30.00,7349599.5,1.229796e+09,716681.94,314676.40,6.0,-0.004943,-0.029656,Cluster 2
1,1,3,1.660222e+12,29.85,0.000469,surge,1,0.000469,0.000469,0.000469,...,precursor,29.90,7307990.0,1.229792e+09,715352.90,314642.84,1.0,0.000402,0.000402,Cluster 2
2,2,5,1.660222e+12,29.87,0.001105,surge,2,0.001607,0.003214,0.003214,...,precursor,29.92,7285245.0,1.229754e+09,714476.10,313576.00,4.0,-0.002773,-0.011090,Cluster 2
3,3,7,1.660222e+12,29.89,0.001610,surge,1,0.001610,0.001610,0.001610,...,precursor,29.88,7318707.0,1.229744e+09,715730.56,313546.60,2.0,-0.000200,-0.000400,Cluster 2
4,4,9,1.660223e+12,29.90,0.001372,surge,1,0.001372,0.001372,0.001372,...,precursor,29.93,7280582.0,1.229794e+09,714140.10,314889.38,7.0,-0.013670,-0.095690,Cluster 2


### what is the efficiency of each cluster?
which variables help describe efficiency?  sum_change and length summarize the sloped area. 

In [11]:

# group by the 'Cluster' column and calculate the product of 'length' and 'sum_change'
grouped = cluster.groupby('Cluster').apply(lambda x: x['length'] * x['sum_change'])

# create the new dataframe with the calculated values
cluster_efficiency = pd.DataFrame({
    'Cluster': grouped.index.get_level_values(0),
    'efficiency': grouped.values
})

print(cluster_efficiency)

        Cluster  efficiency
0     Cluster 0    0.008795
1     Cluster 0    0.040600
2     Cluster 0    0.023038
3     Cluster 0    0.118451
4     Cluster 0    0.058832
...         ...         ...
5566  Cluster 3    0.000483
5567  Cluster 3    0.006031
5568  Cluster 3    0.001530
5569  Cluster 3    0.000564
5570  Cluster 3    0.002981

[5571 rows x 2 columns]


In [13]:
# aggregate the 'efficiency' column by 'Cluster'
aggregated = cluster_efficiency.groupby('Cluster').agg({'efficiency': 'mean'})

print(aggregated)

           efficiency
Cluster              
Cluster 0    0.033680
Cluster 1    0.028678
Cluster 2    0.032486
Cluster 3    0.026289


In [14]:
# convert the 'efficiency' column to a percentage
aggregated['efficiency'] = aggregated['efficiency'].apply(lambda x: '{:.2%}'.format(x))

print(aggregated)

          efficiency
Cluster             
Cluster 0      3.37%
Cluster 1      2.87%
Cluster 2      3.25%
Cluster 3      2.63%


## apply efficiency metric to each row, according to cluster metric

In [16]:
# merge the aggregated dataframe with the original dataframe
merged = pd.merge(cluster, aggregated, on='Cluster')

print(merged)

      Unnamed: 0  group          time   s_MP    change   type  length  \
0              0      1  1.660222e+12  29.99  0.317588  surge       1   
1              1      3  1.660222e+12  29.85  0.000469  surge       1   
2              2      5  1.660222e+12  29.87  0.001105  surge       2   
3              3      7  1.660222e+12  29.89  0.001610  surge       1   
4              4      9  1.660223e+12  29.90  0.001372  surge       1   
...          ...    ...           ...    ...       ...    ...     ...   
5566        5565  11131  1.691438e+12  12.42  0.000483  surge       1   
5567        5566  11133  1.691438e+12  12.43  0.000805  surge       3   
5568        5567  11135  1.691441e+12  12.43  0.001530  surge       1   
5569        5568  11137  1.691442e+12  12.44  0.000564  surge       1   
5570        5569  11139  1.691443e+12  12.41  0.002981  surge       1   

      sum_change    s_area  surge_area  ...   p_MP   p_buyCap      p_askCap  \
0       0.317588  0.317588    0.317588  ... 

### CSV out: categoric efficiency, per cluster

In [17]:
# setup storage string

now = datetime.datetime.now()
# print("Current date and time: ")
print(now.strftime("%Y-%m-%d %H:%M:%S"))

# model_type = 'kmeans'
run_time = now.strftime("%Y-%m-%d %H:%M:%S")
file_name = 'efficiency_metric/' +run_time+ '.csv'
print(file_name)
merged.to_csv(file_name)

2023-08-14 10:37:50
efficiency_metric/2023-08-14 10:37:50.csv
